In [23]:
import pandas as pd
import math

In [24]:
table = pd.read_csv('TABLE.csv', sep =";")
table

,DEPART,ARRIVEE,DISTANCE
0,A,B,3
1,A,C,4
2,A,D,8
3,B,C,4
4,B,E,9
5,C,D,4
6,C,E,5
7,D,E,7
8,D,F,4
9,D,B,6


In [25]:
names = [item for sublist in [list(table.DEPART) , list(table.ARRIVEE)]  for item in sublist]
names = sorted(list(set(names)))

adjmatrix = pd.DataFrame(columns = names)

for m in names:
    for n in names:
        if n in list(table[table["DEPART"] == m].ARRIVEE):
            adjmatrix.loc[m,n] = float(table[(table["DEPART"] == m) & (table["ARRIVEE"] == n)].DISTANCE)
        else:
            adjmatrix.loc[m,n] = 0
            
adjmatrix     

,A,B,C,D,E,F,G
A,0,3,4,8,0,0,0
B,0,0,4,0,9,0,0
C,0,0,0,4,5,0,0
D,0,6,0,0,7,4,0
E,0,0,5,0,0,0,8
F,0,15,0,0,0,0,10
G,0,0,0,0,0,0,0


In [71]:
adlist = []
start = "A"
stop = "G"

for m in names:
    if m == start:
        sublist = [m, float(0), "unvisited",[start]]
    else:
        sublist = [m, math.inf, "unvisited", [start]]
    
    for n in names:
        if adjmatrix.loc[m, n] > 0:
            sublist.append([n, adjmatrix.loc[m, n]])  
    if len(sublist) > 4:
        adlist.append(sublist)
adlist


[['A', 0.0, 'unvisited', ['A'], ['B', 3.0], ['C', 4.0], ['D', 8.0]],
 ['B', inf, 'unvisited', ['A'], ['C', 4.0], ['E', 9.0]],
 ['C', inf, 'unvisited', ['A'], ['D', 4.0], ['E', 5.0]],
 ['D', inf, 'unvisited', ['A'], ['B', 6.0], ['E', 7.0], ['F', 4.0]],
 ['E', inf, 'unvisited', ['A'], ['C', 5.0], ['G', 8.0]],
 ['F', inf, 'unvisited', ['A'], ['B', 15.0], ['G', 10.0]]]

In [72]:
mapper_output = []

for l in adlist:
    new_m = []
    if (l[2] == "unvisited") & (l[1] != math.inf):
        l[2] = "visited"
        if l[0] not in l[3]:
            l[3].append(l[0])
        mapper_output.append(l)
        for m in l[4:]:
            m[1] = m[1] + l[1]
            new_m = m.copy()
            new_m.append("unvisited")
            new_m.append([l[0]])
            mapper_output.append(new_m)
    else:
        mapper_output.append(l)

mapper_output

[['A', 0.0, 'visited', ['A'], ['B', 3.0], ['C', 4.0], ['D', 8.0]],
 ['B', 3.0, 'unvisited', ['A']],
 ['C', 4.0, 'unvisited', ['A']],
 ['D', 8.0, 'unvisited', ['A']],
 ['B', inf, 'unvisited', ['A'], ['C', 4.0], ['E', 9.0]],
 ['C', inf, 'unvisited', ['A'], ['D', 4.0], ['E', 5.0]],
 ['D', inf, 'unvisited', ['A'], ['B', 6.0], ['E', 7.0], ['F', 4.0]],
 ['E', inf, 'unvisited', ['A'], ['C', 5.0], ['G', 8.0]],
 ['F', inf, 'unvisited', ['A'], ['B', 15.0], ['G', 10.0]]]

Shuffle & sort part : on va sort par primary key : le node index (A,B etc.)

In [69]:
# Shuffle & sort:

sorted(mapper_output)

[['A', 0.0, 'visited', ['A'], ['B', 3.0], ['C', 4.0], ['D', 8.0]],
 ['B', 3.0, 'unvisited', 'A'],
 ['B', inf, 'unvisited', ['A'], ['C', 4.0], ['E', 9.0]],
 ['C', 4.0, 'unvisited', 'A'],
 ['C', inf, 'unvisited', ['A'], ['D', 4.0], ['E', 5.0]],
 ['D', 8.0, 'unvisited', 'A'],
 ['D', inf, 'unvisited', ['A'], ['B', 6.0], ['E', 7.0], ['F', 4.0]],
 ['E', inf, 'unvisited', ['A'], ['C', 5.0], ['G', 8.0]],
 ['F', inf, 'unvisited', ['A'], ['B', 15.0], ['G', 10.0]]]

In [27]:
# mapper1 BROUILLON

visited_table = []
current_node = start
current_dist = []

for l in adlist:
    if l[1] != math.inf:
        visited_table.append(l[2:])
    else:
        var_list = []
        for j in range(len(l[2:])):
            var_list. append([l[2:][j][0], math.inf])
        current_dist.append(var_list)
            
print(current_dist)
print(visited_table)

[[['C', inf], ['E', inf]], [['D', inf], ['E', inf]], [['B', inf], ['E', inf], ['F', inf]], [['C', inf], ['G', inf]], [['B', inf], ['G', inf]]]
[[['B', 3.0], ['C', 4.0], ['D', 8.0]]]


In [17]:
[m[0] for m in visited_table[0]]

['B', 'C', 'D']

In [18]:
red_output = []

for l in adlist:
    for m in visited_table[0]:
        if l[0] == m[0]:
            l[1] = m[1]
            for i in range(len(l[2:])):
                l[2:][i][1] = l[2:][i][1] + m[1]
                
    red_output.append(l)

red_output

[['A', 0, ['B', 3.0], ['C', 4.0], ['D', 8.0]],
 ['B', 3.0, ['C', 10.0], ['E', 15.0]],
 ['C', 4.0, ['D', 12.0], ['E', 13.0]],
 ['D', 8.0, ['B', 22.0], ['E', 23.0], ['F', 20.0]],
 ['E', inf, ['C', 5.0], ['G', 8.0]],
 ['F', inf, ['B', 15.0], ['G', 10.0]]]

In [15]:
red_output[2][1] != math.inf

True

In [35]:
## MAPPER 2
# on veut avoir la liste des nodes parcourus, et garder les plus petits pour chaque point à partir du node d'avant

step = 2 # on a parcouru tous les nodes qui étaient à 1 vecteur de start, maintenant on veut append les distances
visited_table = []

for l in red_output:
    if l[1] != math.inf:
        for i in range(len(l[2:])):
            visited_table.append(l[2:][i])
        
visited_table

[['B', 3.0],
 ['C', 4.0],
 ['D', 8.0],
 ['C', 10.0],
 ['E', 15.0],
 ['D', 12.0],
 ['E', 13.0],
 ['B', 22.0],
 ['E', 23.0],
 ['F', 20.0]]